In [ ]:
# Import the libraries
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from skimage.transform import resize
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from functions import (convert_to_binary, select_half_video, extractMobilNetfeatures)

In [ ]:
############################################################
# PARAMETER SETTING
cnn_flag = True # train own CNN 

transfer_MobileNetV2_flag_1 = True 
# fine-tune pretrained MobileNetV2 

r = 56 # target image size        
ne = 200 ###########200 # number of epochs for CNN training
eta = 0.0001 # training rate


Videos = ['Pigs_49651_960_540_500f', 'Koi_5652_952_540',\
          'Pigeons_8234_1280_720','Pigeons_4927_960_540_600f', \
          'Pigeons_29033_960_540_300f']

cl_names = ['Linear Discriminant Analysis', '3-nn', 'Decision Tree',
            'SVM', 'Bagging', 'Random Forest']
###########################################

In [ ]:
##############################################
# CALCULATIONS

data_path = "../data/"
training_fold = 1 # 1 or 2
test_fold = 3 - training_fold

for video in Videos:
    
    print('\n\n#####  ' + video + '  #####\n\n')
    
    folder = os.path.join(data_path, video + "_clips")
    bb_file = os.path.join(data_path, "BB_" + video + ".csv")

    # Read training and testing data (half video; must not be random!)
    imds_1,labels_1 = select_half_video(folder,bb_file,part = training_fold) # training
    imds_2,labels_2 = select_half_video(folder,bb_file,part = test_fold) # testing
    
    ll1 = list(np.sort(np.unique(labels_1)))
    ll2 = list(np.sort(np.unique(labels_2)))
    
    ll = list(np.sort(np.unique(ll1+ll2)))
    n_classes = len(ll1)    

    X_train_im = []
    X_train = []
    y_train_num = []
    for i in range(len(imds_1)):
        y_train_num.append(ll.index(labels_1[i]))
        X_train.append(resize(imds_1[i], (r, r)))
        X_train_im.append(imds_1[i])
    y_train,_ = convert_to_binary(y_train_num,n_classes)
    X_train = np.array(X_train)/255                       

    X_test_im = []
    X_test = []
    y_test_num = [] # numerical testing labels
    for i in range(len(imds_2)):
        if labels_2[i] in set(ll):
            y_test_num.append(ll.index(labels_2[i]))
            X_test.append(resize(imds_2[i], (r, r)))
            X_test_im.append(imds_2[i])

    y_test,_ = convert_to_binary(y_test_num,len(ll))  
    X_test = np.array(X_test)/255    

    print('Training size = ',X_train.shape,' Testing size = ',X_test.shape)
    
    # Augment training data (just in case)
    datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.2, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip = True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

    datagen.fit(X_train)
    
    out_fn = video        

    if cnn_flag:
        # CNN
        #==============================
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(r,r,3)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(n_classes, activation='softmax'))
        #==============================

        opt = Adam(learning_rate = eta)
        model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
        history = model.fit(datagen.flow(X_train,y_train),epochs = ne, verbose = True)

        acc = history.history['accuracy']
        loss = history.history['loss']
        epochs_range = range(ne)
        plt.figure() # figsize=(15, 15))
        plt.subplot(121)
        plt.plot(epochs_range, acc)
        plt.title('Training Accuracy')
        plt.subplot(122)
        plt.plot(epochs_range, loss)
        plt.title('Training Loss')

        predictions = model.predict(X_test) # assigned probabilities
        y_assigned = np.argmax(predictions, axis=-1) # assigned labels

        testing_accuracy = np.mean(y_test_num == y_assigned)
        print('Testing accuracy = ', testing_accuracy)

        # Confusion matrix
        cm = confusion_matrix(y_test_num,y_assigned)
        plt.figure() 
        sns.heatmap(cm, xticklabels = ll, yticklabels = ll)        
        
        # Save results

        TrainingAccLoss = np.hstack((np.reshape(acc,(-1,1)),np.reshape(loss,(-1,1))))
        np.savetxt("TrainingAccLoss_"+out_fn+"_fold"+str(training_fold)+".csv", TrainingAccLoss, delimiter=",")
        
        TestingLabelsAssignedLabels = np.hstack( \
            (np.reshape(y_test_num,(-1,1)),np.reshape(y_assigned,(-1,1))))
        np.savetxt("TestingLabelsAssignedLabels_"+ \
            out_fn+"_fold"+str(training_fold)+".csv", TestingLabelsAssignedLabels, delimiter=",")

    if transfer_MobileNetV2_flag_1:
        #================================================================
        # TRANSFER LEARNING (1) MobileNetV2 - further NN model

        base_model = tf.keras.applications.MobileNetV2(input_shape = \
           (r, r, 3), include_top = False, weights = "imagenet")

        base_model.trainable = False
        model = tf.keras.Sequential([base_model,
             tf.keras.layers.GlobalAveragePooling2D(),
             tf.keras.layers.Dropout(0.2),
             tf.keras.layers.Dense(n_classes, activation="softmax")                                     
            ])
        base_learning_rate = eta
        model.compile(optimizer = \
                      tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics=['accuracy'])

        history = model.fit(datagen.flow(X_train, \
                     y_train),epochs = ne, verbose = True)

        predictions = model.predict(X_test) # assigned probabilities
        y_assigned = np.argmax(predictions, axis=-1) # assigned labels

        testing_accuracy = np.mean(y_test_num == y_assigned)
        print('Testing accuracy MobileNetV2= ', testing_accuracy)
        
        TestingLabelsAssignedLabels = np.hstack( \
            (np.reshape(y_test_num,(-1,1)),np.reshape(y_assigned,(-1,1))))
        np.savetxt("TestingLabelsAssignedLabelsMobileNetV2_"+ \
            out_fn+"_fold"+str(training_fold)+".csv", TestingLabelsAssignedLabels, delimiter=",")     
        
